# 03 — Evaluate Pretrained Baseline (No Fine-Tuning)

**Goal:** Test if the pretrained COCO model works well enough WITHOUT fine-tuning.

**Hypothesis:** Fine-tuning on 377 images (23 real) causes catastrophic forgetting.
The baseline might already be good enough!

**Inputs:**
- `yolov8s-seg.pt` — Pretrained COCO weights (no fine-tuning)
- Real validation images from `data/yolo-seg-board/images/val/`

**Outputs:**
- Visual predictions on real images
- IoU metrics
- Detection rate
- Decision: fine-tune or use baseline as-is?

## 1. Setup

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
from ultralytics import YOLO

# Paths
PROJECT_ROOT = Path.cwd().parent
DATA_DIR = PROJECT_ROOT / "prototyping/data"
DATASET_DIR = DATA_DIR / "yolo-seg-board"
VAL_IMG_DIR = DATASET_DIR / "images" / "val"
VAL_LBL_DIR = DATASET_DIR / "labels" / "val"

# Load PRETRAINED model (NO fine-tuning)
print("Loading pretrained YOLOv8s-seg (COCO weights, no fine-tuning)...")
model = YOLO("yolov8s-seg.pt")
print("✅ Model loaded\n")

## 2. Evaluate on Real Images Only

In [ ]:
# Get ALL validation images (real + synthetic)
all_imgs = sorted(VAL_IMG_DIR.glob("*.jpg"))
real_imgs = [p for p in all_imgs if p.name.startswith("real_")]
synth_imgs = [p for p in all_imgs if p.name.startswith("synth_")]

print(f"Found {len(all_imgs)} total validation images:")
print(f"  - {len(real_imgs)} real")
print(f"  - {len(synth_imgs)} synthetic")

# Predict on ALL images
results = []
for img_path in all_imgs:
    pred = model.predict(str(img_path), imgsz=640, conf=0.25, verbose=False)[0]
    img_type = "real" if img_path.name.startswith("real_") else "synth"
    results.append((img_path, pred, img_type))

print(f"\n✅ Ran predictions on {len(results)} images")

## 3. Visual Inspection

In [ ]:
# Display predictions - sample from both real and synthetic
n_real_display = min(len(real_imgs), 4)
n_synth_display = min(len(synth_imgs), 12)
n_display = n_real_display + n_synth_display

# Get sample results
real_results = [r for r in results if r[2] == "real"][:n_real_display]
synth_results = [r for r in results if r[2] == "synth"][:n_synth_display]
display_results = real_results + synth_results

n_cols = 4
n_rows = (n_display + n_cols - 1) // n_cols

fig, axes = plt.subplots(n_rows, n_cols, figsize=(5 * n_cols, 5 * n_rows))
axes = axes.flatten() if n_display > 1 else [axes]

for idx, (img_path, pred, img_type) in enumerate(display_results):
    annotated = pred.plot()
    axes[idx].imshow(cv2.cvtColor(annotated, cv2.COLOR_BGR2RGB))
    
    # Count detections
    n_det = len(pred.boxes) if pred.boxes is not None else 0
    conf = pred.boxes.conf[0].item() if n_det > 0 else 0.0
    
    # Show detected class
    cls_name = "none"
    if n_det > 0:
        cls_id = int(pred.boxes.cls[0].item())
        cls_name = pred.names[cls_id]
    
    type_label = "[REAL]" if img_type == "real" else "[SYNTH]"
    axes[idx].set_title(f"{type_label} {img_path.name[:20]}\n{cls_name} (conf={conf:.2f})", fontsize=8)
    axes[idx].axis("off")

for idx in range(n_display, len(axes)):
    axes[idx].set_visible(False)

plt.suptitle(f"Pretrained Baseline Predictions (No Fine-Tuning)\n{len(real_results)} real + {len(synth_results)} synthetic samples", fontsize=14, fontweight="bold")
plt.tight_layout()
plt.show()

## 4. Compute IoU Metrics

In [ ]:
def compute_mask_iou(pred_mask, gt_polygon, img_shape):
    """Compute IoU between predicted mask and ground truth polygon."""
    h, w = img_shape[:2]
    
    # Create GT mask
    gt_mask = np.zeros((h, w), dtype=np.uint8)
    cv2.fillPoly(gt_mask, [gt_polygon.reshape(-1, 1, 2)], 1)
    
    # Resize pred mask to image size
    if pred_mask.shape != (h, w):
        pred_mask = cv2.resize(
            pred_mask.astype(np.uint8), (w, h), 
            interpolation=cv2.INTER_NEAREST
        )
    
    # Compute IoU
    intersection = np.logical_and(pred_mask, gt_mask).sum()
    union = np.logical_or(pred_mask, gt_mask).sum()
    return intersection / union if union > 0 else 0.0

# Compute metrics - separate by real vs synthetic
real_ious = []
real_confs = []
real_detected = 0

synth_ious = []
synth_confs = []
synth_detected = 0

for img_path, pred, img_type in results:
    # Load GT label
    lbl_path = VAL_LBL_DIR / f"{img_path.stem}.txt"
    if not lbl_path.exists():
        continue
    
    label_text = lbl_path.read_text().strip().split()
    coords = [float(x) for x in label_text[1:]]
    
    img = cv2.imread(str(img_path))
    h, w = img.shape[:2]
    gt_pts = np.array(
        [[coords[i] * w, coords[i + 1] * h] for i in range(0, len(coords), 2)],
        dtype=np.int32,
    )
    
    # Check if detected
    if pred.masks is not None and len(pred.masks) > 0:
        pred_mask = pred.masks.data[0].cpu().numpy()
        iou = compute_mask_iou(pred_mask, gt_pts, (h, w))
        conf = pred.boxes.conf[0].item()
        
        if img_type == "real":
            real_ious.append(iou)
            real_confs.append(conf)
            real_detected += 1
        else:
            synth_ious.append(iou)
            synth_confs.append(conf)
            synth_detected += 1
    else:
        if img_type == "real":
            real_ious.append(0.0)
            real_confs.append(0.0)
        else:
            synth_ious.append(0.0)
            synth_confs.append(0.0)

# Compute totals
all_ious = real_ious + synth_ious
all_confs = real_confs + synth_confs
total_detected = real_detected + synth_detected
total_images = len(real_ious) + len(synth_ious)

# Print summary
print("\n" + "="*70)
print("BASELINE MODEL PERFORMANCE (NO FINE-TUNING)")
print("="*70)

print(f"\n📊 OVERALL:")
print(f"  Detection Rate: {total_detected}/{total_images} ({100*total_detected/total_images:.1f}%)")
print(f"  Mean IoU: {np.mean(all_ious):.4f}")
print(f"  Mean Confidence: {np.mean([c for c in all_confs if c > 0]):.4f}" if total_detected > 0 else "  Mean Confidence: N/A")
print(f"  IoU > 0.7: {sum(1 for iou in all_ious if iou > 0.7)}/{len(all_ious)} ({100*sum(1 for iou in all_ious if iou > 0.7)/len(all_ious):.1f}%)")

print(f"\n📊 REAL IMAGES ({len(real_ious)} images):")
print(f"  Detection Rate: {real_detected}/{len(real_ious)} ({100*real_detected/len(real_ious):.1f}%)")
print(f"  Mean IoU: {np.mean(real_ious):.4f}")
print(f"  Mean Confidence: {np.mean([c for c in real_confs if c > 0]):.4f}" if real_detected > 0 else "  Mean Confidence: N/A")
print(f"  IoU > 0.7: {sum(1 for iou in real_ious if iou > 0.7)}/{len(real_ious)} ({100*sum(1 for iou in real_ious if iou > 0.7)/len(real_ious):.1f}%)")

print(f"\n📊 SYNTHETIC IMAGES ({len(synth_ious)} images):")
print(f"  Detection Rate: {synth_detected}/{len(synth_ious)} ({100*synth_detected/len(synth_ious):.1f}%)")
print(f"  Mean IoU: {np.mean(synth_ious):.4f}")
print(f"  Mean Confidence: {np.mean([c for c in synth_confs if c > 0]):.4f}" if synth_detected > 0 else "  Mean Confidence: N/A")
print(f"  IoU > 0.7: {sum(1 for iou in synth_ious if iou > 0.7)}/{len(synth_ious)} ({100*sum(1 for iou in synth_ious if iou > 0.7)/len(synth_ious):.1f}%)")

print("\n" + "="*70)
print("COMPARISON TO FINE-TUNED MODELS")
print("="*70)
print(f"Baseline (no tuning):  Overall IoU = {np.mean(all_ious):.4f}, Detection = {100*total_detected/total_images:.1f}%")
print(f"                       Real IoU = {np.mean(real_ious):.4f}, Synth IoU = {np.mean(synth_ious):.4f}")
print(f"Fine-tuned YOLOv8s:    Overall IoU = 0.403,  Real IoU = 0.358, Synth IoU = 0.406")
print(f"Fine-tuned YOLOv8n:    Overall IoU = 0.329,  Real IoU = 0.523, Synth IoU = 0.318")

# Decision logic
if np.mean(all_ious) > 0.7 and total_detected / total_images > 0.8:
    print("\n✅ BASELINE WINS! Skip fine-tuning and use pretrained model.")
elif np.mean(all_ious) > 0.5:
    print("\n⚠️  BASELINE IS DECENT. Consider minimal fine-tuning (freeze=24, lr=0.0001).")
else:
    print("\n❌ BASELINE NEEDS IMPROVEMENT. Collect more real data or try extreme freezing.")

# Histogram comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].hist(real_ious, bins=20, edgecolor='black', alpha=0.7, color='coral')
axes[0].axvline(np.mean(real_ious), color='red', linestyle='--', linewidth=2, label=f'Mean: {np.mean(real_ious):.3f}')
axes[0].set_xlabel('IoU')
axes[0].set_ylabel('Count')
axes[0].set_title(f'Real Images (n={len(real_ious)})')
axes[0].set_xlim(0, 1)
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].hist(synth_ious, bins=20, edgecolor='black', alpha=0.7, color='lightblue')
axes[1].axvline(np.mean(synth_ious), color='blue', linestyle='--', linewidth=2, label=f'Mean: {np.mean(synth_ious):.3f}')
axes[1].set_xlabel('IoU')
axes[1].set_ylabel('Count')
axes[1].set_title(f'Synthetic Images (n={len(synth_ious)})')
axes[1].set_xlim(0, 1)
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.suptitle('IoU Distribution: Real vs Synthetic (Baseline Model)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 5. Check What COCO Class Is Being Detected

In [ ]:
# Count detected classes
from collections import Counter

detected_classes = []
for img_path, pred, img_type in results:
    if pred.boxes is not None and len(pred.boxes) > 0:
        cls_id = int(pred.boxes.cls[0].item())
        cls_name = pred.names[cls_id]
        detected_classes.append(cls_name)

class_counts = Counter(detected_classes)

print("\n🔍 What COCO Classes Are Detected?")
print("="*40)
for cls_name, count in class_counts.most_common():
    print(f"  {cls_name}: {count}x")

print("\n💡 This tells us which COCO features are transferring to boards.")
print("   Common matches: laptop, book, cell phone, keyboard, remote")

## 6. Decision: Fine-Tune or Use Baseline?

**If baseline IoU > 0.7 and detection > 80%:**
- ✅ Use pretrained model as-is (no fine-tuning)
- Save baseline weights for deployment
- Maybe just tune confidence threshold

**If baseline IoU 0.5-0.7:**
- Try extreme freezing: `freeze=24` layers
- Use tiny learning rate: `lr0=0.0001`
- Collect 50+ more real training images

**If baseline IoU < 0.5:**
- Collect 200+ real images
- Try different augmentation
- Consider using a different architecture